*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

Read train and test data into dataframes:

In [ ]:
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['type', 'message']) # training data
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['type', 'message']) # testing data

Change ham and spam into identical numeric values for both dataframes and create training and test labels:

In [ ]:
train_df['type'] = train_df['type'].str.replace('ham','0')
train_df['type'] = train_df['type'].str.replace('spam','1')

test_df['type'] = test_df['type'].str.replace('ham','0')
test_df['type'] = test_df['type'].str.replace('spam','1')

training_labels = np.asarray(train_df.pop('type')).astype('float32').reshape((-1,1))
test_labels = np.asarray(test_df.pop('type')).astype('float32').reshape((-1,1))

Create dateset out of both dataframes:

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_df['message'].values, training_labels)
)

test_dataset = tf.data.Dataset.from_tensor_slices(
    (test_df['message'].values, test_labels)
)

Next shuffle the data for training and create batches:

In [ ]:
BUFFER_SIZE = 100
BATCH_SIZE = 32

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

The raw text needs to be processed before it can be used by the model. A simple way to process text for training is using the TextVectorization layer.

In [ ]:
VOCAB_SIZE = 1000

text_vec = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
text_vec.adapt(train_dataset.map(lambda text, label: text))

Create the model for the classification:

In [ ]:
model = tf.keras.Sequential([
    text_vec,
    tf.keras.layers.Embedding(len(text_vec.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Compile the Keras model to configure the training process:

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Train the model:

In [ ]:
history = model.fit(train_dataset, 
                    epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

Next up is the function which will classify every message as ham or spam:

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = model.predict(np.array([pred_text]))
  print(prediction[0][0])
  return [prediction[0][0], "ham" if prediction[0][0] < 0.5 else "spam"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
